In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json

import pandas as pd

In [14]:
import os

big_data = pd.DataFrame()
## find current folder root
root = os.getcwd()

all_files = glob.iglob(r'downloaded_configs\passivbot_v5.9.x\configs\v5.9.0\harmony\recursive\longshort\harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off' + '/**/result.txt', recursive=True)

files = []
for file in all_files:
    if '\\old\\' not in file:
        files.append(file)

print(files)

i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = os.path.abspath(filename.replace('result.txt', '../config.json'))
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        big_data = big_data.append(d, ignore_index=True)
big_data

['downloaded_configs\\passivbot_v5.9.x\\configs\\v5.9.0\\harmony\\recursive\\longshort\\harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off\\ATOMUSDT_20221222000537_d44da\\ATOMUSDT\\result.txt', 'downloaded_configs\\passivbot_v5.9.x\\configs\\v5.9.0\\harmony\\recursive\\longshort\\harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off\\CHZUSDT_20221222000537_d44da\\CHZUSDT\\result.txt', 'downloaded_configs\\passivbot_v5.9.x\\configs\\v5.9.0\\harmony\\recursive\\longshort\\harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off\\DOGEUSDT_20221222000537_d44da\\DOGEUSDT\\result.txt', 'downloaded_configs\\passivbot_v5.9.x\\configs\\v5.9.0\\harmony\\recursive\\longshort\\harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off\\DOTUSDT_20221222000537_d44da\\DOTUSDT\\result.txt', 'downloaded_configs\\passivbot_v5.9.x\\configs\\v5.9.0\\harmony\\recursive\\longshort\\harmony_recurv_auenabled_longshort_0.25_0.12WE_bwtp_off\\DYDXUSDT_20221222000537_d44da\\DYDXUSDT\\result.txt', 'downloaded_c

C:\Users\skhan\AppData\Local\Temp\ipykernel_14456\126579774.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_14456\126579774.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_14456\126579774.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_14456\126579774.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=Tr

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss
0,downloaded_configs\passivbot_v5.9.x\configs\v5...,ATOMUSDT,True,101.800,0.0688,0.018619,1.060040,100.0,0.8712,3.352779,True,27.510,0.0233,0.016841,0.280098,100.0000,0.9529,5.578458
1,downloaded_configs\passivbot_v5.9.x\configs\v5...,CHZUSDT,True,96.650,0.0982,0.017995,1.279900,100.0,0.8595,3.104862,True,28.440,0.0370,0.015488,0.349507,100.0000,0.9355,3.149304
2,downloaded_configs\passivbot_v5.9.x\configs\v5...,DOGEUSDT,True,144.300,0.1030,0.013416,0.755366,100.0,0.8923,4.440958,True,-111.300,-0.5310,1.730750,6.649780,0.2748,-241.0000,581.450000
3,downloaded_configs\passivbot_v5.9.x\configs\v5...,DOTUSDT,True,99.130,0.0834,0.012744,0.859077,100.0,0.8838,3.695137,True,15.340,0.0168,0.009927,0.119440,100.0000,0.9835,4.614583
4,downloaded_configs\passivbot_v5.9.x\configs\v5...,DYDXUSDT,True,26.210,0.0531,0.020629,0.285243,100.0,0.9444,5.735417,True,27.870,0.0540,0.016606,0.245021,100.0000,0.9610,3.342363
5,downloaded_configs\passivbot_v5.9.x\configs\v5...,EOSUSDT,True,32.040,0.0274,0.027116,1.309660,100.0,0.8581,3.595833,True,3.326,0.0045,0.069210,0.572953,100.0000,0.8382,4.189583
6,downloaded_configs\passivbot_v5.9.x\configs\v5...,HBARUSDT,True,66.160,0.0818,0.009254,0.666551,100.0,0.8999,3.573612,True,25.690,0.0360,0.009242,0.275973,100.0000,0.9542,3.349304
7,downloaded_configs\passivbot_v5.9.x\configs\v5...,LINKUSDT,True,71.170,0.0518,0.017020,0.365803,100.0,0.9329,6.122917,True,18.510,0.0158,0.014173,0.164406,100.0000,0.9762,6.031958
8,downloaded_configs\passivbot_v5.9.x\configs\v5...,MANAUSDT,True,100.200,0.1090,0.017288,1.447380,100.0,0.8520,2.286804,True,19.320,0.0285,0.026497,0.336111,100.0000,0.9391,4.251375
9,downloaded_configs\passivbot_v5.9.x\configs\v5...,MATICUSDT,True,150.100,0.1200,0.019128,0.618375,100.0,0.9043,3.083333,True,18.800,0.0226,0.015581,0.276874,100.0000,0.9536,5.791667


In [15]:
allowed_symbols = [
    "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
    "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
    "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
    "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
    "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
    "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
    "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
]
#allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df.to_csv('5.9.0.csv')
df

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,adgt_%
0,downloaded_configs\passivbot_v5.9.x\configs\v5...,ATOMUSDT,True,101.80,0.0688,0.018619,1.060040,100.0,0.8712,3.352779,True,27.510,0.0233,0.016841,0.280098,100.0000,0.9529,5.578458,0.0921
1,downloaded_configs\passivbot_v5.9.x\configs\v5...,CHZUSDT,True,96.65,0.0982,0.017995,1.279900,100.0,0.8595,3.104862,True,28.440,0.0370,0.015488,0.349507,100.0000,0.9355,3.149304,0.1352
2,downloaded_configs\passivbot_v5.9.x\configs\v5...,DOGEUSDT,True,144.30,0.1030,0.013416,0.755366,100.0,0.8923,4.440958,True,-111.300,-0.5310,1.730750,6.649780,0.2748,-241.0000,581.450000,-0.4280
3,downloaded_configs\passivbot_v5.9.x\configs\v5...,DOTUSDT,True,99.13,0.0834,0.012744,0.859077,100.0,0.8838,3.695137,True,15.340,0.0168,0.009927,0.119440,100.0000,0.9835,4.614583,0.1002
5,downloaded_configs\passivbot_v5.9.x\configs\v5...,EOSUSDT,True,32.04,0.0274,0.027116,1.309660,100.0,0.8581,3.595833,True,3.326,0.0045,0.069210,0.572953,100.0000,0.8382,4.189583,0.0319
6,downloaded_configs\passivbot_v5.9.x\configs\v5...,HBARUSDT,True,66.16,0.0818,0.009254,0.666551,100.0,0.8999,3.573612,True,25.690,0.0360,0.009242,0.275973,100.0000,0.9542,3.349304,0.1178
7,downloaded_configs\passivbot_v5.9.x\configs\v5...,LINKUSDT,True,71.17,0.0518,0.017020,0.365803,100.0,0.9329,6.122917,True,18.510,0.0158,0.014173,0.164406,100.0000,0.9762,6.031958,0.0676
8,downloaded_configs\passivbot_v5.9.x\configs\v5...,MANAUSDT,True,100.20,0.1090,0.017288,1.447380,100.0,0.8520,2.286804,True,19.320,0.0285,0.026497,0.336111,100.0000,0.9391,4.251375,0.1375
9,downloaded_configs\passivbot_v5.9.x\configs\v5...,MATICUSDT,True,150.10,0.1200,0.019128,0.618375,100.0,0.9043,3.083333,True,18.800,0.0226,0.015581,0.276874,100.0000,0.9536,5.791667,0.1426
11,downloaded_configs\passivbot_v5.9.x\configs\v5...,SANDUSDT,True,56.41,0.0667,0.013576,0.629215,100.0,0.9033,4.159721,True,14.840,0.0225,0.108080,0.655966,100.0000,0.7670,12.613875,0.0892


In [16]:
df.drop_duplicates(
    subset=['s',  'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file',  's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp\ipykernel_14456\994885060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,downloaded_configs\passivbot_v5.9.x\configs\v5...,ATOMUSDT,0.0921,0.0688,0.0233,100.0,100.0000,0.8712,0.9529,3.352779,1.060040,0.018619,5.578458,0.280098,0.016841
1,downloaded_configs\passivbot_v5.9.x\configs\v5...,CHZUSDT,0.1352,0.0982,0.0370,100.0,100.0000,0.8595,0.9355,3.104862,1.279900,0.017995,3.149304,0.349507,0.015488
2,downloaded_configs\passivbot_v5.9.x\configs\v5...,DOGEUSDT,-0.4280,0.1030,-0.5310,100.0,0.2748,0.8923,-241.0000,4.440958,0.755366,0.013416,581.450000,6.649780,1.730750
3,downloaded_configs\passivbot_v5.9.x\configs\v5...,DOTUSDT,0.1002,0.0834,0.0168,100.0,100.0000,0.8838,0.9835,3.695137,0.859077,0.012744,4.614583,0.119440,0.009927
5,downloaded_configs\passivbot_v5.9.x\configs\v5...,EOSUSDT,0.0319,0.0274,0.0045,100.0,100.0000,0.8581,0.8382,3.595833,1.309660,0.027116,4.189583,0.572953,0.069210
6,downloaded_configs\passivbot_v5.9.x\configs\v5...,HBARUSDT,0.1178,0.0818,0.0360,100.0,100.0000,0.8999,0.9542,3.573612,0.666551,0.009254,3.349304,0.275973,0.009242
7,downloaded_configs\passivbot_v5.9.x\configs\v5...,LINKUSDT,0.0676,0.0518,0.0158,100.0,100.0000,0.9329,0.9762,6.122917,0.365803,0.017020,6.031958,0.164406,0.014173
8,downloaded_configs\passivbot_v5.9.x\configs\v5...,MANAUSDT,0.1375,0.1090,0.0285,100.0,100.0000,0.8520,0.9391,2.286804,1.447380,0.017288,4.251375,0.336111,0.026497
9,downloaded_configs\passivbot_v5.9.x\configs\v5...,MATICUSDT,0.1426,0.1200,0.0226,100.0,100.0000,0.9043,0.9536,3.083333,0.618375,0.019128,5.791667,0.276874,0.015581
11,downloaded_configs\passivbot_v5.9.x\configs\v5...,SANDUSDT,0.0892,0.0667,0.0225,100.0,100.0000,0.9033,0.7670,4.159721,0.629215,0.013576,12.613875,0.655966,0.108080


In [17]:
dl_proposed = dx[
    #(dx['lm'] == 'n') &
    (dx['bnkrl_%'] == 100) &
    (dx['bnkrs_%'] == 100) &
    # (dx['ebrl'] > 0.9) &  #&
    # (dx['ebrs'] > 0.9) &  #&
    # (dx['adgl_%'] > 0.04) 
    # (dx['pastdl'] < 0.03) &
    # (dx['pastds'] < 0.03) &  #
    # (dx['maxsl'] < 10) &
    # (dx['maxss'] < 10) &
    (dx['adgt_%'].notnull()) &
    (dx['adgt_%'] > 0)
    ]
#dl_proposed = dl_proposed[['file','s','adgl_%', 'bnkrl_%', 'ebrl',  'maxsl', 'pamaxl','pastdl' ]]
dl_proposed

,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
0,downloaded_configs\passivbot_v5.9.x\configs\v5...,ATOMUSDT,0.0921,0.0688,0.0233,100.0,100.0,0.8712,0.9529,3.352779,1.060040,0.018619,5.578458,0.280098,0.016841
1,downloaded_configs\passivbot_v5.9.x\configs\v5...,CHZUSDT,0.1352,0.0982,0.0370,100.0,100.0,0.8595,0.9355,3.104862,1.279900,0.017995,3.149304,0.349507,0.015488
3,downloaded_configs\passivbot_v5.9.x\configs\v5...,DOTUSDT,0.1002,0.0834,0.0168,100.0,100.0,0.8838,0.9835,3.695137,0.859077,0.012744,4.614583,0.119440,0.009927
5,downloaded_configs\passivbot_v5.9.x\configs\v5...,EOSUSDT,0.0319,0.0274,0.0045,100.0,100.0,0.8581,0.8382,3.595833,1.309660,0.027116,4.189583,0.572953,0.069210
6,downloaded_configs\passivbot_v5.9.x\configs\v5...,HBARUSDT,0.1178,0.0818,0.0360,100.0,100.0,0.8999,0.9542,3.573612,0.666551,0.009254,3.349304,0.275973,0.009242
7,downloaded_configs\passivbot_v5.9.x\configs\v5...,LINKUSDT,0.0676,0.0518,0.0158,100.0,100.0,0.9329,0.9762,6.122917,0.365803,0.017020,6.031958,0.164406,0.014173
8,downloaded_configs\passivbot_v5.9.x\configs\v5...,MANAUSDT,0.1375,0.1090,0.0285,100.0,100.0,0.8520,0.9391,2.286804,1.447380,0.017288,4.251375,0.336111,0.026497
9,downloaded_configs\passivbot_v5.9.x\configs\v5...,MATICUSDT,0.1426,0.1200,0.0226,100.0,100.0,0.9043,0.9536,3.083333,0.618375,0.019128,5.791667,0.276874,0.015581
11,downloaded_configs\passivbot_v5.9.x\configs\v5...,SANDUSDT,0.0892,0.0667,0.0225,100.0,100.0,0.9033,0.7670,4.159721,0.629215,0.013576,12.613875,0.655966,0.108080
12,downloaded_configs\passivbot_v5.9.x\configs\v5...,SUSHIUSDT,0.1124,0.0765,0.0359,100.0,100.0,0.8725,0.9327,1.398612,1.037720,0.019618,3.212500,0.359026,0.017543
